# Current

## Create test example

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import string, random, toolz, pandas, importlib, sys, os

In [3]:
terms = [(term, 1) for term in str(random.choices(string.ascii_letters, k=10**7)).split('Z')]

In [4]:
pandas.Series(len(term) for term, _ in terms).describe()

count    192160.000000
mean        259.199839
std         256.922932
min           4.000000
25%          74.000000
50%         179.000000
75%         359.000000
max        3174.000000
dtype: float64

## Run current version

In [5]:
!git stash
!git checkout main
#importlib.reload(dimod)

Saved working directory and index state WIP on binary-polynomial: bc4521f added symbolic arithmetic refactored relabel_variables, to_binary, to_spin filter out 0 coefficient terms in polynomial construction
Switched to branch 'main'
Your branch is up to date with 'origin/main'.


In [6]:
from dimod import BinaryPolynomial as BPold
import dimod

In [7]:
BPold.__init__??

Signature: BPold.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms
        self._terms = terms = {}
        for term, bias in poly:

            fsterm = asfrozenset(term)

            # when SPIN-valued, s^2 == 1, so we need to handle that case
            # in BINARY, x^2 == x
            if len(fsterm) < len(term) and vartype is Vartype.SPIN:
                new = set()
                term = tuple(term)  # make sure it has .count
                for v in fsterm:
                    if term.count(v) % 2:
                        new.add(v)
                fsterm = frozenset(new)

            if fsterm in terms:
                terms[fsterm] += bias
            else:
                terms[fsterm] = bias

        self.vartype = 

In [8]:
%%prun -s cumtime -l 20
bp = BPold(terms, 'SPIN')

         9711530 function calls (9711450 primitive calls) in 48.033 seconds

   Ordered by: cumulative time
   List reduced from 47 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   48.033   48.033 {built-in method builtins.exec}
        1    0.000    0.000   48.033   48.033 <string>:1(<module>)
        1    0.000    0.000   48.033   48.033 decorators.py:289(new_f)
        1    7.909    7.909   48.033   48.033 polynomial.py:89(__init__)
  5482062   38.220    0.000   38.220    0.000 {method 'count' of 'tuple' objects}
   192160    1.075    0.000    1.135    0.000 polynomial.py:32(asfrozenset)
  3460625    0.690    0.000    0.690    0.000 {method 'add' of 'set' objects}
   384322    0.079    0.000    0.079    0.000 {built-in method builtins.len}
   192170    0.060    0.000    0.060    0.000 {built-in method builtins.isinstance}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
    

In [9]:
pandas.DataFrame((len(term), bias) for term, bias in bp.items()).describe()

,0,1
count,181561.000000,181561.000000
mean,18.931703,1.058377
std,7.552280,8.721284
min,1.000000,1.000000
25%,13.000000,1.000000
50%,20.000000,1.000000
75%,25.000000,1.000000
max,42.000000,3677.000000


In [10]:
term = string.ascii_letters[:12]

In [11]:
%timeit BPold({term: 1}, 'SPIN').to_binary().to_spin() == BPold({term: 1}, 'SPIN')
BPold({term: 1}, 'SPIN').to_binary().to_spin() == BPold({term: 1}, 'SPIN')

1.14 s ± 310 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [12]:
%timeit BPold({term: 1}, 'BINARY').to_spin().to_binary() == BPold({term: 1}, 'BINARY')
BPold({term: 1}, 'BINARY').to_spin().to_binary() == BPold({term: 1}, 'BINARY')

1.22 s ± 265 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [13]:
len(BPold({term: 1}, 'SPIN').to_binary().to_spin())

4096

In [14]:
len(BPold({term: 1}, 'BINARY').to_spin().to_binary())

4096

In [15]:
prun -s cumtime -l 20 bp2 = bp.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         19196037 function calls (19196032 primitive calls) in 21.734 seconds

   Ordered by: cumulative time
   List reduced from 66 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   21.734   21.734 {built-in method builtins.exec}
        1    0.000    0.000   21.733   21.733 <string>:1(<module>)
      2/1   15.297    7.648   21.733   21.733 polynomial.py:232(relabel_variables)
  7237640    2.147    0.000    2.887    0.000 polynomial.py:256(<genexpr>)
   544686    1.648    0.000    2.116    0.000 _collections_abc.py:742(__iter__)
  6874522    0.740    0.000    0.740    0.000 {method 'get' of 'dict' objects}
   363120    0.462    0.000    0.608    0.000 polynomial.py:158(__setitem__)
        1    0.000    0.000    0.517    0.517 polynomial.py:177(copy)
        1    0.000    0.000    0.517    0.517 decorators.py:289(new_f)
        1    0.200    0.200    0.517    0.517 polynomial.py:89(__init__)
  1452484 

In [16]:
prun -s cumtime -l 20 bp3 = bp2.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         19196034 function calls (19196029 primitive calls) in 18.191 seconds

   Ordered by: cumulative time
   List reduced from 63 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   18.191   18.191 {built-in method builtins.exec}
        1    0.000    0.000   18.191   18.191 <string>:1(<module>)
      2/1    9.913    4.957   18.191   18.191 polynomial.py:232(relabel_variables)
  7237640    2.937    0.000    3.940    0.000 polynomial.py:256(<genexpr>)
   544686    1.989    0.000    2.508    0.000 _collections_abc.py:742(__iter__)
  6874522    1.003    0.000    1.003    0.000 {method 'get' of 'dict' objects}
   363120    0.674    0.000    0.886    0.000 polynomial.py:158(__setitem__)
  1452484    0.430    0.000    0.592    0.000 polynomial.py:32(asfrozenset)
   363120    0.404    0.000    0.565    0.000 polynomial.py:120(__delitem__)
   544683    0.350    0.000    0.519    0.000 polynomial.py:149(__getit

In [17]:
bp == bp2

False

In [18]:
bp == bp3

True

In [19]:
bp2 == bp3

False

# Exit interpreter
I just can't get module reloading to work properly here ...

In [20]:
print('BYE')
exit()
raise SystemExit
print('HI')

BYE


SystemExit: 

/home/daniel/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Proposed

## Create test example

In [1]:
import string, random, toolz, pandas, importlib, sys, os, cytoolz, toolz, dimod

In [2]:
import itertools as it

In [3]:
%time terms = [(term, 1) for term in ''.join(random.choices(string.ascii_letters, k=10**6)).split('Z')]

CPU times: user 96.8 ms, sys: 9.77 ms, total: 107 ms
Wall time: 106 ms


In [4]:
pandas.Series(len(term) for term, _ in terms).describe()

count    18962.000000
mean        51.737106
std         52.324975
min          0.000000
25%         14.000000
50%         36.000000
75%         72.000000
max        535.000000
dtype: float64

## Run proposed version

In [5]:
!mv polynomial-perf.ipynb{,.bak}
!git checkout binary-polynomial
!git stash pop
#!touch -c ./dimod/higherorder/polynomial.py
#importlib.reload(dimod)

D	polynomial-perf.ipynb
Already on 'binary-polynomial'
Your branch is ahead of 'origin/binary-polynomial' by 1 commit.
  (use "git push" to publish your local commits)
No stash entries found.


In [6]:
from dimod import BinaryPolynomial as BPnew

In [7]:
BPnew.__init__??

Signature: BPnew.__init__(self, poly, vartype)
Docstring: Initialize self.  See help(type(self)) for accurate signature.
Source:   
    @vartype_argument('vartype')
    def __init__(self, poly, vartype):
        if isinstance(poly, abc.Mapping):
            poly = poly.items()

        # we need to aggregate the repeated terms

        self._terms = terms = defaultdict(int)
        for term, bias in poly:
            terms[freeze_term(term, vartype)] += bias
        
        self.vartype = vartype
File:      ~/dimod/dimod/higherorder/polynomial.py
Type:      function


In [8]:
%%prun -s cumtime -l 20 
bp = BPnew(terms, 'SPIN')

         399010 function calls (398930 primitive calls) in 0.179 seconds

   Ordered by: cumulative time
   List reduced from 46 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.179    0.179 {built-in method builtins.exec}
        1    0.000    0.000    0.179    0.179 <string>:1(<module>)
        1    0.000    0.000    0.178    0.178 decorators.py:289(new_f)
        1    0.013    0.013    0.178    0.178 polynomial.py:108(__init__)
    18962    0.123    0.000    0.165    0.000 polynomial.py:40(freeze_term)
   341921    0.039    0.000    0.039    0.000 polynomial.py:47(<genexpr>)
    18972    0.001    0.000    0.002    0.000 {built-in method builtins.isinstance}
    18963    0.001    0.000    0.001    0.000 {method 'items' of 'dict' objects}
        1    0.000    0.000    0.000    0.000 abc.py:96(__instancecheck__)
        1    0.000    0.000    0.000    0.000 {built-in method _abc._abc_instancecheck}


In [9]:
pandas.DataFrame((len(term), bias) for term, bias in bp.items()).describe()

,0,1
count,18203.000000,18203.000000
mean,17.718838,1.041696
std,7.690828,2.906779
min,0.000000,1.000000
25%,12.000000,1.000000
50%,19.000000,1.000000
75%,24.000000,1.000000
max,39.000000,390.000000


In [10]:
term = string.ascii_letters[:12]

In [11]:
%timeit BPnew({term: 1}, 'SPIN').to_binary().to_spin() == BPnew({term: 1}, 'SPIN')
BPnew({term: 1}, 'SPIN').to_binary().to_spin() == BPnew({term: 1}, 'SPIN')

528 ms ± 55.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [12]:
%timeit BPnew({term: 1}, 'BINARY').to_spin().to_binary() == BPnew({term: 1}, 'BINARY')
BPnew({term: 1}, 'BINARY').to_spin().to_binary() == BPnew({term: 1}, 'BINARY')

793 ms ± 77.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


True

In [13]:
len(BPnew({term: 1}, 'BINARY').to_spin().to_binary())

4096

In [14]:
len(BPnew({term: 1}, 'SPIN').to_binary().to_spin())

4096

In [15]:
#len(BPnew({term: 1}, 'BINARY').to_spin() * BPnew({term: 1}, 'BINARY').to_spin())

In [16]:
prun -s cumtime -l 20 bp2 = bp.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         1799999 function calls (1799994 primitive calls) in 0.717 seconds

   Ordered by: cumulative time
   List reduced from 67 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.717    0.717 {built-in method builtins.exec}
        1    0.000    0.000    0.717    0.717 <string>:1(<module>)
      2/1    0.370    0.185    0.717    0.717 polynomial.py:236(relabel_variables)
   681478    0.127    0.000    0.169    0.000 polynomial.py:260(<genexpr>)
    54612    0.036    0.000    0.074    0.000 _collections_abc.py:742(__iter__)
        1    0.000    0.000    0.046    0.046 polynomial.py:181(copy)
        1    0.000    0.000    0.046    0.046 decorators.py:289(new_f)
        1    0.017    0.017    0.046    0.046 polynomial.py:108(__init__)
   645076    0.042    0.000    0.042    0.000 {method 'get' of 'dict' objects}
    36404    0.031    0.000    0.040    0.000 polynomial.py:162(__setitem__)
    54609   

In [17]:
prun -s cumtime -l 20 bp3 = bp2.relabel_variables(dict(zip(string.ascii_letters, reversed(string.ascii_letters))), inplace=False)

         1799996 function calls (1799991 primitive calls) in 0.425 seconds

   Ordered by: cumulative time
   List reduced from 64 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.425    0.425 {built-in method builtins.exec}
        1    0.000    0.000    0.425    0.425 <string>:1(<module>)
      2/1    0.165    0.082    0.425    0.425 polynomial.py:236(relabel_variables)
   681478    0.106    0.000    0.141    0.000 polynomial.py:260(<genexpr>)
    54612    0.025    0.000    0.053    0.000 _collections_abc.py:742(__iter__)
   645076    0.036    0.000    0.036    0.000 {method 'get' of 'dict' objects}
        1    0.000    0.000    0.029    0.029 polynomial.py:181(copy)
        1    0.000    0.000    0.029    0.029 decorators.py:289(new_f)
        1    0.008    0.008    0.029    0.029 polynomial.py:108(__init__)
    54609    0.019    0.000    0.028    0.000 polynomial.py:153(__getitem__)
    36404   

In [18]:
bp == bp2

False

In [19]:
bp == bp3

True

In [20]:
bp2 == bp3

False

In [21]:
import numpy as np
%load_ext line_profiler

In [22]:
#%lprun -fdimod.make_quadratic  bqm = dimod.make_quadratic(bp, vartype = 'BINARY', strength = 2**6)

In [23]:
import functools as ft
import itertools as it
import cytoolz as tl
from dimod.higherorder.utils import _new_product, as_vartype
from collections import defaultdict

In [24]:
%%time
idx = defaultdict(
    dict,
    tl.valmap(
        tl.compose(dict, ft.partial(map, tl.second)),
        tl.groupby(
            tl.first,
            (
                (frozenset(pair), item)
                for item in bp.items()
                if len(term := tl.first(item)) >= 3
                for pair in it.combinations(term, 2) ))))

CPU times: user 11.4 s, sys: 571 ms, total: 12 s
Wall time: 12 s


In [25]:
len(idx)

1275

In [26]:
#idx

In [27]:
#bp

In [28]:
#terms

In [29]:
que = tl.valmap(tl.compose(set, ft.partial(map, tl.first)), tl.groupby(tl.second, tl.valmap(len, idx).items()))
most = max(que)
most

2679

In [30]:
que[most].pop()

frozenset({'K', 'u'})

In [31]:
que[most]

set()

In [32]:
constraints = []
reduced_terms = [item for item in bp.items() if len(tl.first(item)) <= 2]
pair, terms = tl.topk(1, idx.items(), tl.compose(len,tl.second))[0]
p = _new_product(bp.variables, *pair)
constraints.append((pair, p))
del idx[pair]
pset = {p}

In [33]:
for term, bias in terms.items():
    for pair2 in map(frozenset, (it.combinations(term, 2))):
        if pair2 != pair:
            del idx[pair2][term]
    base_term = term - pair
    new_term = base_term | pset
    if len(new_term) <= 3:
        reduced_terms.append((new_term, bias))
    else:
        for pair3 in map(frozenset, it.combinations(new_term, 2)):
            idx[pair3][new_term] = bias

In [34]:
#reduced_terms

In [35]:
constraints

[(frozenset({'K', 'u'}), 'u*K')]

In [36]:
#terms

In [37]:
tl.topk(5, tl.valmap(len, idx).items(), tl.second)

((frozenset({'V', 's'}), 2668),
 (frozenset({'d', 't'}), 2658),
 (frozenset({'U', 'V'}), 2656),
 (frozenset({'M', 'V'}), 2655),
 (frozenset({'C', 'V'}), 2649))

In [38]:
#%prun -s cumtime -l 30 dimod.reduced_terms, constraints = reduce_terms(bp)

In [39]:
#len(reduced_terms), len(constraints)

In [40]:
%prun -s cumtime -l 30 bqm = dimod.make_quadratic(bp, 1000, 'BINARY')

         67534938 function calls (67534930 primitive calls) in 45.651 seconds

   Ordered by: cumulative time
   List reduced from 104 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   45.651   45.651 {built-in method builtins.exec}
        1    0.023    0.023   45.651   45.651 <string>:1(<module>)
        1    0.624    0.624   45.628   45.628 utils.py:176(make_quadratic)
        1   22.136   22.136   40.487   40.487 utils.py:120(reduce_terms)
 25594299    8.849    0.000    8.849    0.000 utils.py:113(_remove_old)
  5860386    4.291    0.000    6.749    0.000 utils.py:103(_decrement_count)
  3180742    2.057    0.000    2.057    0.000 utils.py:164(<genexpr>)
    58153    0.083    0.000    1.989    0.000 utils.py:62(_binary_product)
    58154    0.087    0.000    1.906    0.000 binary_quadratic_model.py:93(__init__)
    58153    0.270    0.000    1.799    0.000 binary_quadratic_model.py:136(_init_componen

In [41]:
zz = set(range(4))

In [42]:
%lprun -fdimod.higherorder.utils.reduce_terms bqm = dimod.make_quadratic(BPnew({term: 1}, 'SPIN'), strength=5, vartype='SPIN')

Timer unit: 1e-06 s

Total time: 0.006932 s
File: /home/daniel/dimod/dimod/higherorder/utils.py
Function: reduce_terms at line 120

Line #      Hits         Time  Per Hit   % Time  Line Contents
   120                                           def reduce_terms(bp):
   121         1          4.0      4.0      0.1      variables = bp.variables
   122         1          1.0      1.0      0.0      constraints = []
   123                                           
   124         1          0.0      0.0      0.0      reduced_terms = []
   125         1          1.0      1.0      0.0      idx = defaultdict(dict)
   126         2         11.0      5.5      0.2      for item in bp.items():
   127         1          0.0      0.0      0.0          term, bias = item
   128         1          1.0      1.0      0.0          if len(term) <= 2:
   129                                                       reduced_terms.append(item)
   130                                                   else:
   131  

In [43]:
%lprun -fdimod.higherorder.utils.reduce_terms bqm = dimod.make_quadratic(bp, strength=5, vartype=bp.vartype)

Timer unit: 1e-06 s

Total time: 101.475 s
File: /home/daniel/dimod/dimod/higherorder/utils.py
Function: reduce_terms at line 120

Line #      Hits         Time  Per Hit   % Time  Line Contents
   120                                           def reduce_terms(bp):
   121         1       8869.0   8869.0      0.0      variables = bp.variables
   122         1          3.0      3.0      0.0      constraints = []
   123                                           
   124         1          1.0      1.0      0.0      reduced_terms = []
   125         1         29.0     29.0      0.0      idx = defaultdict(dict)
   126     18204      52791.0      2.9      0.1      for item in bp.items():
   127     18203       9428.0      0.5      0.0          term, bias = item
   128     18203      12144.0      0.7      0.0          if len(term) <= 2:
   129       386        823.0      2.1      0.0              reduced_terms.append(item)
   130                                                   else:
   131   

In [44]:
%time len(dimod.make_quadratic(BPnew({term: 1}, 'SPIN'), strength=5, vartype='SPIN'))

CPU times: user 2.84 ms, sys: 13 µs, total: 2.85 ms
Wall time: 2.86 ms


62

In [45]:
make_quadratic??

Object `make_quadratic` not found.


In [46]:
BinaryPolynomial.

SyntaxError: invalid syntax (3330683553.py, line 1)

In [ ]:
dimod.BQM

In [ ]:
itertools

In [ ]:
%prun -s cumtime -l 20 dimod.make_quadratic(bp, 100, vartype='BINARY')

In [ ]:
from dimod.higherorder import utils

In [ ]:
%pwd

In [ ]:
bool(set())

In [ ]:
bool